In [1]:
import base64

import os
import requests
import re
import math
import zipfile
import sys
import logging

import json

from datetime import datetime, timedelta
from os.path import expanduser
from __future__ import division

Data Provider Configuration for Data Robot REST API Plugin
http://help.otus.com/en/articles/2829276-finding-the-powerschool-client-credentials

In [2]:
endpoint = {"table_name":"cc", "query_expression":"termid=ge=3000;termid=lt=3100",
     "projection":"dateleft,schoolid,termid,period_obsolete,attendance_type_code,unused2,currentabsences,currenttardies,attendance,teacherid,lastgradeupdate,section_number,course_number,origsectionid,unused3,teachercomment,lastattmod,asmtscores,firstattdate,finalgrades,studyear,log,expression,studentsectenrl_guid,teacherprivatenote,ab_course_cmp_fun_flg,ab_course_cmp_ext_crd,ab_course_cmp_met_cd,ab_course_eva_pro_cd,ab_course_cmp_sta_cd,dcid,id,studentid,sectionid,dateenrolled"}
    
base_url = 'https://kippchicago.powerschool.com'
table_name = endpoint['table_name']
query = 'q={}&'.format(endpoint['query_expression'])
maxpagesize = 1000
table_columns = endpoint['projection']
ps_client_id = "7d2606ef-c59c-493b-811e-12252b1e2794"
ps_client_secret = "7f7ff316-0535-4f36-a63f-f4c40d54a462"

credentials_concat = '{0}:{1}'.format(ps_client_id, ps_client_secret)
credentials_encoded = base64.b64encode(credentials_concat.encode('utf-8'))

access_token_timestamp = datetime.now()
access_headers = {
        'Authorization': b'Basic ' + credentials_encoded,
        'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
    }
access_payload = {'grant_type':'client_credentials'}

In [3]:
def get_access_token(base_url, credentials_encoded):
    """
    retrieve a new access_token from PowerSchool
        - base_url                  full URL of PS instance (default from CONFIG)
        - credentials_encoded       API credentials (default from CONFIG)
    """
    logging.info('Retrieving new access token')
    access_token_timestamp = datetime.now()
    access_headers = {
            'Authorization': b'Basic ' + credentials_encoded,
            'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
        }
    access_payload = {'grant_type':'client_credentials'}
    r_access = requests.post('{0}/oauth/access_token/'.format(base_url), headers=access_headers, params=access_payload)

    access_json = r_access.json()
    access_json['timestamp'] = str(access_token_timestamp)
    return access_json

In [4]:
access_token = get_access_token(base_url, credentials_encoded)
auth_headers = {'Authorization': 'Bearer {0}'.format(access_token['access_token']),}

In [5]:
r_count = requests.get('{0}/ws/schema/table/{1}/count?{2}'.format(base_url, 
                                                                  table_name, 
                                                                  query), headers=auth_headers)

In [11]:
r_status

200

In [7]:
r_count = requests.get('{0}/ws/schema/table/{1}/count?{2}'.format(base_url, 
                                                                  table_name, 
                                                                  query), 
                       headers=auth_headers)

r_status = r_count.status_code
if r_status != 200:
     logging.info('Response NOT successful. I got code {} '.format(r_status))
     raise ValueError('Response NOT successful. I got code {} '.format(r_status))
else:
     logging.info('Response  successful! I got code {} '.format(r_status))

count_json = r_count.json()
row_count = count_json['count']

pages = int(math.ceil(row_count / maxpagesize))

In [8]:
data = []
for p in range(pages):
    page_number = p + 1

    endpoint = '{0}/ws/schema/table/{1}?{2}page={3}&pagesize={4}&projection={5}'.format(base_url, table_name, query, page_number, maxpagesize, table_columns)
    r_data = requests.get(endpoint, headers=auth_headers)

    if r_data.ok:
        data_json = r_data.json()
        records = data_json['record']
        for r in records:
            data.append(r['tables'][table_name])

In [9]:
save_dir = "~/code"
filename = "current_cc_data.json"
def save_file(save_dir, filename, data):
    """
    check if save folder exists (create if not) and save data to specified filepath
        - filepath
        - data
    """
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)

    filepath = '{0}/{1}'.format(save_dir, filename)
    with open(filepath, 'w+') as outfile:
        json.dump(data, outfile)

    zipfilepath = filepath.replace('.json','.zip')
    with zipfile.ZipFile(zipfilepath, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.write(filepath)

In [10]:
save_file(".", filename, data)